# **Uncertainty Project -- Deep Learning**

---

_Fabio TOCCO, Antoine GUIDON, Yelman YAHI, Anis OUEDGHIRI, Ram NADER_


# Imports


In [ ]:
import os
import random
from typing import Literal
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import tools

import torch
import torch.nn as nn
from torch.utils.data import random_split

import torchvision.datasets as datasets
from torchvision import transforms
from torchvision.transforms import functional as TF

# Setup


In [ ]:
DATA_ROOT = os.path.join(os.path.pardir, "data")
MODELS_ROOT = os.path.join(os.path.pardir, "models")

# Create the directories if they do not exist
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(MODELS_ROOT, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Selected device: {DEVICE}")

# Hyperparameters (DO NOT CHANGE)


In [ ]:
EPOCHS: int = 3
CRITERION: nn.Module = nn.CrossEntropyLoss()
LEARNING_RATE: float = 1e-4
WEIGHT_DECAY: float = 1e-4
BATCH_SIZE: int = 1024

NUM_WORKERS: int = (os.cpu_count() or 0) // 2
print(f"NUM_WORKERS: {NUM_WORKERS}")

# Parameters (change for different training)


In [ ]:
RESIZE_VALUE: int = 32
NORMALIZATION: Literal["MNIST", "ImageNet"] = "MNIST"
SEED: int = 0
SHUFFLE: bool = False

FORCE_RETRAIN: bool = False

tools.seed_everything(seed=SEED)

# Datasets


In [ ]:
data_transforms = tools.get_data_transforms(
    data_root=DATA_ROOT, resize_value=RESIZE_VALUE, normalization=NORMALIZATION
)

train_data = datasets.MNIST(
    DATA_ROOT,
    train=True,
    download=True,
    transform=data_transforms,
)
print(f"Number of train samples: {len(train_data)}")

test_data = datasets.MNIST(
    DATA_ROOT,
    train=False,
    download=True,
    transform=data_transforms,
)
print(f"Number of test samples: {len(test_data)}")

num_classes: int = len(train_data.classes)

# Define the validation set by splitting the training data into 2 subsets (80% training and 20% validation)
n_train_samples = int(len(train_data) * 0.8)
n_validation_samples = len(train_data) - n_train_samples
train_data, validation_data = random_split(
    train_data, [n_train_samples, n_validation_samples]
)

# Experience #1


## DataLoaders


In [ ]:
train_loader, validation_loader, test_loader = tools.get_loaders(
    train_data,
    validation_data,
    test_data,
    shuffle=SHUFFLE,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

## Version 1 - Random weights


### Pre-Training


In [ ]:
PRETRAINED: bool = False
model_version_1 = tools.make_resnet18(num_classes, pretrained=PRETRAINED)

OPTIMIZER = torch.optim.Adam(
    model_version_1.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

model_name = tools.get_model_name(
    pretrained=PRETRAINED, shuffle=SHUFFLE, seed=SEED, normalization=NORMALIZATION
)
print(f"Model name: {model_name}")
model_dir = os.path.join(MODELS_ROOT, model_name)
os.makedirs(model_dir, exist_ok=True)

config = {
    "model": "resnet18",
    "pretrained": PRETRAINED,
    "shuffle": SHUFFLE,
    "seed": SEED,
    "normalization": NORMALIZATION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "optimizer": "Adam",
    "criterion": "CrossEntropyLoss",
    "num_train_samples": len(train_data),
    "num_val_samples": len(validation_data),
    "num_test_samples": len(test_data),
}

### Training loop


In [ ]:
model_version_1, _, _, _ = tools.train_model(
    model=model_version_1,
    train_loader=train_loader,
    validation_loader=validation_loader,
    criterion=CRITERION,
    optimizer=OPTIMIZER,
    epochs=EPOCHS,
    device=DEVICE,
    file_path=os.path.join(model_dir, model_name + ".pt"),
    verbose=True,
    save_plots=True,
    config=config,
)

test_loss, test_accuracy = tools.evaluate(
    model_version_1, test_loader, criterion=CRITERION, device=DEVICE
)

print(
    f"{model_name} -- Loss on test set: {test_loss:.4f} | Accuracy on test set: {100 * test_accuracy:.2f}%",
)

## Version 2 - Pre-trained weights on ImageNet


### Pre-Training


In [ ]:
PRETRAINED: bool = True
model_version_2 = tools.make_resnet18(num_classes, pretrained=PRETRAINED)

OPTIMIZER = torch.optim.Adam(
    model_version_2.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

model_name = tools.get_model_name(
    pretrained=PRETRAINED, shuffle=SHUFFLE, seed=SEED, normalization=NORMALIZATION
)
print(f"Model name: {model_name}")
model_dir = os.path.join(MODELS_ROOT, model_name)
os.makedirs(model_dir, exist_ok=True)

config = {
    "model": "resnet18",
    "pretrained": PRETRAINED,
    "shuffle": SHUFFLE,
    "seed": SEED,
    "normalization": NORMALIZATION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "weight_decay": WEIGHT_DECAY,
    "optimizer": "Adam",
    "criterion": "CrossEntropyLoss",
    "num_train_samples": len(train_data),
    "num_val_samples": len(validation_data),
    "num_test_samples": len(test_data),
}

### Training loop


In [ ]:
model_version_2, _, _, _ = tools.train_model(
    model=model_version_2,
    train_loader=train_loader,
    validation_loader=validation_loader,
    criterion=CRITERION,
    optimizer=OPTIMIZER,
    epochs=EPOCHS,
    device=DEVICE,
    file_path=os.path.join(model_dir, model_name + ".pt"),
    verbose=True,
    save_plots=True,
    config=config,
)

test_loss, test_accuracy = tools.evaluate(
    model_version_2, test_loader, criterion=CRITERION, device=DEVICE
)

print(
    f"{model_name} -- Loss on test set: {test_loss:.4f} | Accuracy on test set: {100 * test_accuracy:.2f}%",
)

# Experience #2


## DataLoaders


In [ ]:
SEED = 0
NUM_SAMPLES: int = 20

train_loader, validation_loader, test_loader = tools.get_loaders(
    train_data,
    validation_data,
    test_data,
    shuffle=SHUFFLE,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

tools.visualize_predictions(
    model_version_2, test_data, device=DEVICE, num_samples=NUM_SAMPLES, seed=SEED
)

# Experience #3


In [ ]:
SEED = 0
tools.seed_everything(seed=SEED)

train_loader, validation_loader, test_loader = tools.get_loaders(
    train_data,
    validation_data,
    test_data,
    shuffle=SHUFFLE,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

# ----------------------------------------------------------------------------
# ENTRAÎNER OU CHARGER LES 7 MODÈLES (AUTOMATIQUE)
# ----------------------------------------------------------------------------

NUM_MODELS: int = 7
PRETRAINED: bool = False

models, model_paths = tools.load_or_train_ensemble(
    num_models=NUM_MODELS,
    num_classes=num_classes,
    train_loader=train_loader,
    validation_loader=validation_loader,
    test_loader=test_loader,
    criterion=CRITERION,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
    device=DEVICE,
    models_root=MODELS_ROOT,
    pretrained=PRETRAINED,
    shuffle=SHUFFLE,
    normalization=NORMALIZATION,
    force_retrain=FORCE_RETRAIN,  # True pour forcer le réentraînement
    partial_load=True,  # True pour charger les modèles existants
    verbose=True,
)

print(f"\n✓ Ensemble prêt avec {len(models)} modèles!")
print(f"\nChemins des modèles:")
for i, path in enumerate(model_paths, 1):
    print(f"  {i}. {path}")

# Afficher les performances sur le test set
print(f"\n{'=' * 70}")
print("PERFORMANCES SUR LE TEST SET")
print(f"{'=' * 70}\n")

for i, (model, path) in enumerate(zip(models, model_paths), 1):
    test_loss, test_accuracy = tools.evaluate(
        model, test_loader, criterion=CRITERION, device=DEVICE
    )
    model_name = Path(path).parent.name
    print(
        f"Modèle {i}/{NUM_MODELS} ({model_name}): "
        f"Loss = {test_loss:.4f} | Accuracy = {test_accuracy * 100:.2f}%"
    )

print(f"\n{'=' * 70}\n")


SEED = 0
tools.seed_everything(seed=SEED)

samples, sel_idx = tools.get_random_samples(
    test_data, set_size=len(test_data), seed=SEED, num_samples=20
)

# Extraire les images et labels
imgs = torch.stack([img for img, _ in samples])  # Shape: (20, 3, 32, 32)
labels = torch.tensor([y for _, y in samples])  # Shape: (20,)

print(f"Indices sélectionnés: {sel_idx}")
print(f"Batch shape: {imgs.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Labels: {labels.tolist()}\n")

BLUR_LEVELS: list[float] = [0.5, 1.0, 1.5, 2.0, 2.5]

for sigma in BLUR_LEVELS:
    blurred_imgs = TF.gaussian_blur(
        imgs, kernel_size=3, sigma=sigma
    )  # Shape: (20, 3, 32, 32)

# Experience #4


# Experience #5


In [ ]:
SEED = 0

tools.seed_everything(seed=SEED)

train_loader, validation_loader, test_loader = tools.get_loaders(
    train_data,
    validation_data,
    test_data,
    shuffle=SHUFFLE,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

# ----------------------------------------------------------------------------
# 1. CHARGER OU ENTRAÎNER LES 7 MODÈLES (AUTOMATIQUE)
# ----------------------------------------------------------------------------

NUM_MODELS: int = 7
PRETRAINED: bool = False

FORCE_RETRAIN: bool = True

models, model_paths = tools.load_or_train_ensemble(
    num_models=NUM_MODELS,
    num_classes=num_classes,
    train_loader=train_loader,
    validation_loader=validation_loader,
    test_loader=test_loader,
    criterion=CRITERION,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
    device=DEVICE,
    models_root=MODELS_ROOT,
    pretrained=PRETRAINED,
    shuffle=SHUFFLE,
    normalization=NORMALIZATION,
    force_retrain=FORCE_RETRAIN,
    partial_load=True,
    verbose=True,
)

print(f"\n✓ Ensemble prêt avec {len(models)} modèles!")
print(f"Chemins des modèles:")
for i, path in enumerate(model_paths, 1):
    print(f"  {i}. {path}")


# ----------------------------------------------------------------------------
# 2. SÉLECTIONNER 20 IMAGES ALÉATOIRES DU TEST SET
# ----------------------------------------------------------------------------

ANGLES: list[int] = list(range(0, 361, 10))  # 0, 10, ..., 360
NUM_SAMPLES: int = 20
SEED = 0

tools.seed_everything(seed=SEED)

samples, sel_idx = tools.get_random_samples(
    test_data, set_size=len(test_data), seed=SEED, num_samples=NUM_SAMPLES
)

# Extraire les images et labels
imgs = torch.stack([img for img, _ in samples])  # Shape: (20, 3, 32, 32)
labels = torch.tensor([y for _, y in samples])  # Shape: (20,)

print(f"Indices sélectionnés: {sel_idx}")
print(f"Batch shape: {imgs.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Labels: {labels.tolist()}\n")


# ----------------------------------------------------------------------------
# 3. SUIVRE 4 IMAGES SPÉCIFIQUES POUR ANALYSE DÉTAILLÉE
# ----------------------------------------------------------------------------


track_indices = sel_idx[:4]  # Les 4 premières images
per_image_data = {
    idx: {
        "label": int(labels[i]),  # Label vrai
        "angle_probs": {},  # Probs moyennes par angle
        "predictions": {},  # Prédiction par angle
    }
    for i, idx in enumerate(track_indices)
}

print(f"Images suivies pour analyse détaillée: {track_indices}")
print(f"Labels vrais: {[per_image_data[idx]['label'] for idx in track_indices]}\n")


# ----------------------------------------------------------------------------
# 4. BOUCLE SUR LES ANGLES - VERSION VECTORISÉE ✅
# ----------------------------------------------------------------------------


print("Angle | Accuracy (moyenne softmax sur 7 modèles)")
print("-" * 45)

accuracies = []  # Stocker les accuracies pour plot

for angle in ANGLES:
    # Rotation du batch complet
    rot_imgs = TF.rotate(
        imgs,
        angle,
        interpolation=transforms.InterpolationMode.BILINEAR,
        # fill=0,
    )  # Shape: (20, 3, 32, 32)

    # Prédictions moyennes sur les 7 modèles (VERSION VECTORISÉE)
    mean_probs = tools.get_mean_probs_fast(
        rot_imgs, models, device=DEVICE
    )  # Shape: (20, 10) ✅

    # Prédictions pour chaque image (VERSION VECTORISÉE)
    preds = mean_probs.argmax(dim=1)  # Shape: (20,) ✅

    # Accuracy sur les 20 images
    correct = (preds.cpu() == labels).sum().item()
    acc = correct / NUM_SAMPLES
    accuracies.append(acc)

    print(f"{angle:5d}° | {acc:.2%}")

    # Stockage des probas pour les 4 images suivies
    for i, idx in enumerate(track_indices):
        per_image_data[idx]["angle_probs"][angle] = mean_probs[i].cpu().clone()
        per_image_data[idx]["predictions"][angle] = preds[i].item()

print(f"\n✓ Évaluation terminée sur {len(ANGLES)} angles de rotation.\n")


# ----------------------------------------------------------------------------
# 5. AFFICHAGE DÉTAILLÉ DES 4 IMAGES SUIVIES
# ----------------------------------------------------------------------------


show_angles = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]

print("\n" + "=" * 70)
print("APERÇU DÉTAILLÉ DES PROBABILITÉS MOYENNES (4 images suivies)")
print("=" * 70)

for idx in track_indices:
    true_y = per_image_data[idx]["label"]
    print(f"\n📸 Image idx={idx} | Vrai label: {true_y}")
    print("-" * 60)
    print(" Angle | Pred | P(pred) | P(vrai) | Correct?")
    print("-" * 60)

    for angle in show_angles:
        if angle not in per_image_data[idx]["angle_probs"]:
            continue

        probs = per_image_data[idx]["angle_probs"][angle]
        pred = per_image_data[idx]["predictions"][angle]

        p_pred = probs[pred].item()
        p_true = probs[true_y].item()
        is_correct = "✓" if pred == true_y else "✗"

        print(f"{angle:6d}° | {pred:4d} | {p_pred:7.3f} | {p_true:7.3f} | {is_correct}")


# ----------------------------------------------------------------------------
# 6. VISUALISATION 1: ACCURACY VS ANGLE
# ----------------------------------------------------------------------------


plt.figure(figsize=(12, 6))
plt.plot(ANGLES, accuracies, marker="o", linewidth=2, markersize=6, color="steelblue")
plt.axhline(y=1.0, color="green", linestyle="--", alpha=0.3, label="Perfect accuracy")
plt.xlabel("Angle de rotation (°)", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.title(
    "Robustesse de l'ensemble de modèles aux rotations", fontsize=14, fontweight="bold"
)
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim(0, 1.05)
plt.tight_layout()
plt.show()

print(f"\n📊 Accuracy min: {min(accuracies):.2%} | Accuracy max: {max(accuracies):.2%}")
print(f"📊 Accuracy moyenne: {np.mean(accuracies):.2%}\n")

In [ ]:
# ----------------------------------------------------------------------------
# 7. VISUALISATION 2: PROBABILITÉ DE LA VRAIE CLASSE PAR IMAGE
# ----------------------------------------------------------------------------

plt.figure(figsize=(14, 7))

for idx in track_indices:
    label = per_image_data[idx]["label"]
    angles = sorted(per_image_data[idx]["angle_probs"].keys())

    # Probabilité de la vraie classe pour chaque angle
    probs_true = [per_image_data[idx]["angle_probs"][a][label].item() for a in angles]

    plt.plot(
        angles,
        probs_true,
        marker="o",
        label=f"Chiffre {label} (idx={idx})",
        linewidth=2,
    )

plt.xlabel("Angle de rotation (°)", fontsize=12)
plt.ylabel("P(vraie classe)", fontsize=12)
plt.title(
    "Confiance du modèle sur la vraie classe vs angle de rotation",
    fontsize=14,
    fontweight="bold",
)
plt.legend(title="Vrai label", fontsize=10)
plt.grid(True, alpha=0.3)
plt.ylim(0, 1.05)
plt.axhline(y=0.5, color="red", linestyle="--", alpha=0.3, label="Seuil 50%")
plt.tight_layout()
plt.show()

# ----------------------------------------------------------------------------
# 8. VISUALISATION 3: HEATMAP DES PRÉDICTIONS PAR IMAGE ET ANGLE
# ----------------------------------------------------------------------------

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, idx in enumerate(track_indices):
    ax = axes[i]
    label = per_image_data[idx]["label"]

    # Matrice (angle, classe) des probabilités
    angles = sorted(per_image_data[idx]["angle_probs"].keys())
    prob_matrix = np.array(
        [per_image_data[idx]["angle_probs"][a].numpy() for a in angles]
    )  # Shape: (len(angles), 10)

    im = ax.imshow(prob_matrix.T, aspect="auto", cmap="viridis", vmin=0, vmax=1)

    ax.set_xlabel("Angle (index)", fontsize=10)
    ax.set_ylabel("Classe", fontsize=10)
    ax.set_title(f"Image {idx} | Vrai label: {label}", fontsize=12, fontweight="bold")
    ax.set_yticks(range(10))
    ax.set_xticks(range(0, len(angles), 5))
    ax.set_xticklabels([angles[i] for i in range(0, len(angles), 5)])

    # Highlight de la vraie classe
    ax.axhline(y=label, color="red", linestyle="--", linewidth=2, alpha=0.5)

    plt.colorbar(im, ax=ax, label="Probabilité")

plt.suptitle(
    "Heatmap des probabilités par classe et angle de rotation",
    fontsize=14,
    fontweight="bold",
    y=0.995,
)
plt.tight_layout()
plt.show()

print("\n✅ Analyse complète terminée!")